## Bike Rental Data Management Project

## Library and Data Imports

In [1]:
import glob
import pandas as pd

## Citi Bike Data Overview

The Citi Bike data is stored in monthly files. Let's read those in and concatenate them in a single dataframe.

In [2]:
files = glob.glob("../data/raw/JC-2016*-citibike-tripdata.csv")
df_list = []

for filename in files:
  data = pd.read_csv(filename)
  df_list.append(data)
df_bike = pd.concat(df_list)

# reset the index after concatenation
df_bike.reset_index(drop=True, inplace=True)

# add an ID column -- this will be useful for a primary key
df_bike['id'] = df_bike.index

Let's visually inspect the beginning and end of the dataset

In [3]:
df_bike.head(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,0
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,1


In [4]:
df_bike.tail(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
247582,2424,2016-12-31 23:44:50,2017-01-01 00:25:14,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,26219,Subscriber,1960.0,2,247582
247583,2419,2016-12-31 23:44:50,2017-01-01 00:25:10,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,24471,Subscriber,1956.0,1,247583


Some initial observations:

- checking the data dictionary:
    - `Trip Duration` is in seconds
    - `Gender` can be `0=unknown`, `1=male`, and `2=female`
    - `User Type` has values `Customer=24 hour pass or 3 day user` and `Subscriber=Annual Member`

let's check next for duplicated rows.

In [5]:
duplicates = df_bike.duplicated()
duplicates.value_counts()

False    247584
dtype: int64

Let's continue exploring by looking at numeric fields

In [6]:
df_bike.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,id
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534,123791.500000
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687,71471.488861
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000,61895.750000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000,123791.500000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000,185687.250000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000,247583.000000


A couple observations:
- `Trip Duration` jumps five orders of magnitude from the 75th percentile to the maximum
- `Birth Year` has a minimum value of 1900, which would correspond to an age of 116

It also looks like `Birth Year` might be missing some values

In [7]:
# checking for missing values
df_bike.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
id                             0
dtype: int64

Let's determine the percentage of missing data in `Birth Year`....just 7.67 %

In [9]:
(1-(df_bike['Birth Year'].count()/df_bike['Bike ID'].count()))*100

7.673759208995734

let's finish initial exploration by checking data types

In [10]:
df_bike.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
id                           int64
dtype: object

`Start Time` and `Stop Time` need to be datetimes

In [11]:
# casting objects as timestamps
df_bike['Start Time'] = pd.to_datetime(df_bike['Start Time'],infer_datetime_format=True)
df_bike['Stop Time'] = pd.to_datetime(df_bike['Stop Time'],infer_datetime_format=True)
df_bike.dtypes

Trip Duration                       int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth Year                        float64
Gender                              int64
id                                  int64
dtype: object

## Cleaning and Transforming Citibike Data

It is convenient to estimate age from the birth year

In [ ]:
df_bike['Age']=2016-df_bike['Birth Year']